In [1]:
import os
import string
#import argparse

import numpy as np
import tensorflow as tf

/Users/florianmuellerklein/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/florianmuellerklein/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading saved models for inference

Loading a saved inference graph is very similar to loading up a saved session. Breaking down the procedure into steps we have:

1. Load the graph definition with tensorflow
2. Make the loaded graph the default graph for the session
3. Retrieve the input placeholders and output tensors from the graph
4. Push data through the graph using a `feed_dict` like normal

In this example we will build a pig latin translation program using the saved inference graph for the encoder-decoder network. 

### Load the inference graph

Tensorflow provides file I/O capabilities within its `gfile` module. We can use that or the standard `with open('filename.pb', 'rb') as f:` methodology. They should both work for our case. If you happen to be using 'Google Cloud Storage' then you will have to use `gfile` rather than the conventional python methods. 

After reading the graph definition file we will load it in a way that allows us to interact with it by using `tf.import_graph_def()`. Which takes the parsed graph definition as it's first argument and a name. The name will be a prefix that is added to each of the names in the graph. For example, if we had something called 'outputs' in our original graph it will now be called 'name/outputs'. 

In [2]:
with tf.gfile.GFile('./saved_translator/piglatin_enc-dec.pb', 'rb') as f:
    graph_definition = tf.GraphDef()
    graph_definition.ParseFromString(f.read())
    
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_definition, name='encoder-decoder')

### Viewing objects in the graph

Sometimes you will load a graph that you didn't make, so you may have to look at all of the objects in the graph. We can loop through and print everything. Notice how everything now has our prefix added to all of the names. 

In [3]:
with tf.Session(graph=graph) as sess:
    for op in tf.get_default_graph().get_operations():
        print(op.name)

encoder-decoder/encoder_input
encoder-decoder/embeddings
encoder-decoder/embeddings/read
encoder-decoder/input_embedding/embedding_lookup
encoder-decoder/encoder/rnn/Rank
encoder-decoder/encoder/rnn/range/start
encoder-decoder/encoder/rnn/range/delta
encoder-decoder/encoder/rnn/range
encoder-decoder/encoder/rnn/concat/values_0
encoder-decoder/encoder/rnn/concat/axis
encoder-decoder/encoder/rnn/concat
encoder-decoder/encoder/rnn/transpose
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_1
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat/axis
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros/Const
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros
encoder-decoder/encoder/rnn/DropoutWrapperZeroState/Basic

### Using the graph

Now that we've loaded the graph and seen all of the objects in it we can use it to make inferences on some data. We need only to remember the names for the input and output nodes of our TensorFlow graph. In our case they are `encoder-decoder/encoder_input` and `encoder-decoder/ecoder_pred`. We will load those from the graph and they will be used exactly like the placeholders and outputs that we've set up previously during training

In [5]:
with tf.Session(graph=graph) as sess:
    # by convention we need to add the `:0` to the names, this gets us the actual tensor
    word_input = graph.get_tensor_by_name('encoder-decoder/encoder_input:0')
    predictions = graph.get_tensor_by_name('encoder-decoder/decoder_pred:0')

## Make an interactive english to pig latin translator

Lets allow the user to enter some input and have the network translate it.

In [6]:
eng_phrase = input('Enter a phrase to translate: ')

Enter a phrase to translate: translate this into pig latin please


### Do the translation

In [9]:
chars = string.ascii_letters + '>_'
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for i, ch in enumerate(chars)}

with tf.Session(graph=graph) as sess:
    word_input = graph.get_tensor_by_name('encoder-decoder/encoder_input:0')
    predictions = graph.get_tensor_by_name('encoder-decoder/decoder_pred:0')

    output_translation = []
    for word in eng_phrase.split():

        val_input = [char2idx[c] for c in word]
        val_input = np.asarray(val_input).reshape(1, len(val_input))

        # for the inference mode we only pass the english word to translate
        prediction = sess.run(predictions, feed_dict={word_input: val_input})


        output_translation.append(''.join([idx2char[idx] for idx in prediction[0]]))

    print(' '.join(output_translation).replace('_', ''))

tanrsatleay histsay igorcay igcay atilnsay pelaseay
